In [1]:
import sys
sys.path.append('..')


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score, precision_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
import pickle 


In [3]:
df = pd.read_csv("../datasets/original/dataset.csv")

df.head(1)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print("Total rows: ", df.shape[0])

Total rows:  4920


## Data Preprocessig

In [26]:
df['Disease'].value_counts()

Allergy                                    120
GERD                                       120
Jaundice                                   120
Osteoarthristis                            120
Malaria                                    120
Typhoid                                    120
Gastroenteritis                            120
Hypothyroidism                             120
Urinary tract infection                    120
Tuberculosis                               120
Acne                                       120
Fungal infection                           120
Common Cold                                120
Pneumonia                                  120
AIDS                                       120
Arthritis                                  120
Hepatitis E                                120
Drug Reaction                              120
Hepatitis D                                120
Peptic ulcer diseae                        120
Dimorphic hemmorhoids(piles)               120
Paralysis (br

In [6]:
df.describe()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
count,4920,4920,4920,4920,4572,3714,2934,2268,1944,1692,1512,1194,744,504,306,240,192,72
unique,41,34,48,54,50,38,32,26,21,22,21,18,11,8,4,3,3,1
top,Allergy,vomiting,vomiting,fatigue,high_fever,headache,nausea,abdominal_pain,abdominal_pain,yellowing_of_eyes,yellowing_of_eyes,irritability,malaise,abnormal_menstruation,chest_pain,chest_pain,blood_in_sputum,muscle_pain
freq,120,822,870,726,378,348,390,264,276,228,198,120,126,72,96,144,72,72


In [7]:
df.isna().sum()


Disease          0
Symptom_1        0
Symptom_2        0
Symptom_3        0
Symptom_4      348
Symptom_5     1206
Symptom_6     1986
Symptom_7     2652
Symptom_8     2976
Symptom_9     3228
Symptom_10    3408
Symptom_11    3726
Symptom_12    4176
Symptom_13    4416
Symptom_14    4614
Symptom_15    4680
Symptom_16    4728
Symptom_17    4848
dtype: int64

In [8]:
df.isnull().sum()


Disease          0
Symptom_1        0
Symptom_2        0
Symptom_3        0
Symptom_4      348
Symptom_5     1206
Symptom_6     1986
Symptom_7     2652
Symptom_8     2976
Symptom_9     3228
Symptom_10    3408
Symptom_11    3726
Symptom_12    4176
Symptom_13    4416
Symptom_14    4614
Symptom_15    4680
Symptom_16    4728
Symptom_17    4848
dtype: int64

In [9]:
cols = df.columns
data = df[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)

df = df.fillna(0)
df.head()


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,itching,skin_rash,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df_serverity = pd.read_csv("../datasets/original/symptom-severity.csv")
df_serverity.head()


,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [11]:
vals = df.values
symptoms = df_serverity['Symptom'].unique()

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = df_serverity[df_serverity['Symptom']
                                             == symptoms[i]]['weight'].values[0]

d = pd.DataFrame(vals, columns=df.columns)
d = d.replace('dischromic _patches', 0)
d = d.replace('spotting_ urination', 0)
df = d.replace('foul_smell_of urine', 0)

df.head()


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Training

In [12]:
data = df.iloc[:, 1:].values
labels = df['Disease'].values


In [13]:
# take 20% of data to train
x_train, x_test, y_train, y_test = train_test_split(
    data, labels, train_size=0.2, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(984, 17) (3936, 17) (984,) (3936,)


## Evaluation

In [14]:
tree = DecisionTreeClassifier(criterion='gini', random_state=42, max_depth=13)
tree.fit(x_train, y_train)
preds = tree.predict(x_test)
conf_mat = confusion_matrix(y_test, preds)
df_cm = pd.DataFrame(
    conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())
print('Accuracy =', accuracy_score(y_test, preds)*100)
print('F1-score =', f1_score(y_test, preds, average='macro')*100)


Accuracy = 97.66260162601627
F1-score = 97.6590167411519


In [15]:
conf_mat = confusion_matrix(y_test, preds)
df_cm = pd.DataFrame(
    conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())
print('Accuracy =', accuracy_score(y_test, preds)*100)
print('F1-score =', f1_score(y_test, preds, average='macro')*100)


Accuracy = 97.66260162601627
F1-score = 97.6590167411519


In [16]:
print('Accuracy =', accuracy_score(y_test, preds)*100)
print("Recall =", recall_score(y_test, preds, average='macro')*100)
print("Precision =", precision_score(y_test, preds, average='macro')*100)
print('F1-score =', f1_score(y_test, preds, average='macro')*100)

Accuracy = 97.66260162601627
Recall = 97.67383401228145
Precision = 97.89201292669904
F1-score = 97.6590167411519


In [17]:
pickle.dump(tree, open('../models/model.pkl', 'wb'))
